In [46]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

df = pd.read_csv('pairwise_52seconds_share.csv')

G = nx.Graph()

for _, row in df.iterrows():
    user_1 = f"user_{row['userID_1']}"
    user_2 = f"user_{row['userID_2']}"
    video_node = f"video_{row['videoID']}"
    
    if not G.has_node(user_1):
        G.add_node(user_1, type='user', color='blue')
    
    if not G.has_node(user_2):
        G.add_node(user_2, type='user', color='blue')
        
    if not G.has_node(video_node):
        G.add_node(video_node, type='video', color='green')
        
    G.add_edge(user_1, video_node, timestamp=row['timestamp_1'])
    G.add_edge(user_2, video_node, timestamp=row['timestamp_2'])
    G.add_edge(user_2, user_1, timestamp=row['timestamp_1'])
    
G.remove_edges_from(nx.selfloop_edges(G))

In [3]:
nx.write_gexf(G, "blue_helm.gexf")


In [6]:
n = G.number_of_nodes()
m = G.number_of_edges()

k = round((2 * m) / n)

H = nx.watts_strogatz_graph(n= n, k = k, p = 0.1)

nx.write_gexf(H, "watts_strogatz_random.gexf")

In [5]:

m_par = int(m/n)

K = nx.barabasi_albert_graph(n = n, m = m_par)

nx.write_gexf(K, "barabasi_albert.gexf")

In [50]:
def transitivity(net):
    return nx.transitivity(net)

def rich_club(net):
    return nx.rich_club_coefficient(net, normalized=True)

def average_degree(G):
    return sum(dict(G.degree()).values()) / G.number_of_nodes()

def deg_assortativity(G):
    return nx.degree_assortativity_coefficient(G)

In [49]:
nx.is_connected(G)

False

In [51]:
def compare_networks(net, base_networks):
    metrics = [transitivity, average_degree, deg_assortativity]
    metric_names = [metric.__name__ for metric in metrics]
    
    results = {metric_name: [] for metric_name in metric_names}
    networks = [('blue_helm', net)] + [(f'baseline{i+1}', base_net) for i, base_net in enumerate(base_networks)]
    
    for name, network in networks:
        for metric in metrics:
            results[metric.__name__].append(metric(network))
            
    return results
    

In [52]:
result = compare_networks(G, [H, K])

In [53]:
result

{'transitivity': [0.010092078570901202, 0.3533781785925488, 0],
 'average_degree': [3.8392857142857144, 4.0, 1.9995421245421245],
 'deg_assortativity': [-0.14743884690490577,
  -0.020316945640402123,
  -0.0699597735998525]}